In [1]:
## Learning while working LSTM models

# Cross Entropy Loss function works well with Label Encoder whereaas Binary Cross Entorpy is used for multi class classification.


# RNN LSTM Model for Binary Classification

In [5]:
 #Import All Libraries Here
import pandas as pd
from sklearn.metrics import accuracy_score ,  roc_curve, auc , classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np

import gensim
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split , RandomizedSearchCV
import xgboost as xgb 
import statistics
from scipy.fftpack import fft, ifft


from imblearn.over_sampling import SMOTE 
from imblearn.over_sampling import RandomOverSampler

from igraph import *

from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
import optuna

import sklearn.datasets

# PyTorch Import

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


In [9]:
# Record Constants
INPUT_TRAIN_IN = "../../../data/train_in.csv"
INPUT_TRAIN_OUT = "../../../data/train_out.csv"  
INPUT_TEST_IN = "../../../data/test_in.csv"
INPUT_TEST_OUT = "../../../data/test_out.csv"
INPUT_VALIDATION_IN = "../../../data/valid_in_nucleo.csv"
INPUT_VALIDATION_OUT  = "../../../data/valid_out.csv"

TARGET_MODEL_PATH = '../../webapp/model_files'


WINDOW_SIZE =  25 # Final RNA Sequence will be 101 Length 
MAX_LENGTH = (WINDOW_SIZE*2) +1

# 1 - One Hot Encoding with Pytorch in build Emnedding 
ENCODING_METHOD = 1

# 1 - LSTM with Cross Entropy 
MODEL = 1

FRAMEWORK = "PYTORCH"


PERFORM_DATA_BALANCING = True
K_MERS_SIZE = 3


# Startegy to Crop Sequene
# MID - Modification is present at Mid of cropped Sequence 
# END - Modification is present at End of cropepd Sequence 
CROP_STRATEGY = 'END'

# Y Category Encoding Method
# LABEL or ONE_HOT 
TARGET_ENCODING = 'LABEL'


In [10]:
#Read X Varaibles and Y Varaibles

x_train_raw =  pd.read_csv(INPUT_TRAIN_IN, header=None , skiprows=1 , nrows= 2 )
y_train_raw =  pd.read_csv(INPUT_TRAIN_OUT, header=None , skiprows=1 , nrows= 2)

x_test_raw =  pd.read_csv(INPUT_TEST_IN, header=None , skiprows=1 )
y_test_raw =  pd.read_csv(INPUT_TEST_OUT, header=None , skiprows=1)

x_valid_raw =  pd.read_csv(INPUT_VALIDATION_IN, header=None , skiprows=1 )
y_valid_raw =  pd.read_csv(INPUT_VALIDATION_OUT, header=None , skiprows=1 )


middle_index = (x_train_raw.shape[1] // 2) + 1

if CROP_STRATEGY == 'MID':
    STRAT_INEDX =middle_index - WINDOW_SIZE -1 
    END_INDEX =middle_index + WINDOW_SIZE 

if CROP_STRATEGY == 'END':
    STRAT_INEDX =middle_index - (WINDOW_SIZE*2) -1 
    END_INDEX =middle_index


In [ ]:
print(f"Train Shape of X : {x_train_raw.shape} and Tranin Shape of Y : {x_train_raw.shape}")
print(f"Test Shape of X : {x_test_raw.shape} and Test Shape of Y : {y_test_raw.shape}")
print(f"Validation Shape of X : {x_valid_raw.shape} and Validation Shape of Y : {y_valid_raw.shape}")

Train Shape of X : (304661, 1001) and Tranin Shape of Y : (304661, 1001)
Test Shape of X : (1200, 1001) and Test Shape of Y : (1200, 12)
Validation Shape of X : (3599, 1001) and Validation Shape of Y : (3599, 12)


In [12]:
## Filter Dataset to Keep only Target Binary Class 

RMs = ['hAm','hCm','hGm','hTm','hm1A','hm5C','hm5U','hm6A','hm6Am','hm7G','hPsi','Atol','NonMoD']
RMEncoding = [12,1,2,3,4,5,6,7,8,9,10,11,0]

def convert_y_to_original_labels(row):
    label = ""
    for index , n in enumerate(row.tolist()) :
        if n == 1 :
            label = RMs[index]
    if label == '':
        return 'NonMoD'
    return label

def prepare_data_for_binary_classification(x_data , y_data , prediction_class):
    # Convert One Hot Encoded Y to to Original Labels 
    y_original_labels = y_data.apply(convert_y_to_original_labels,axis=1)
    x_data['Label'] = y_original_labels
    target_class = [prediction_class, 'NonMoD']      
    selected_rna_data = x_data[x_data['Label'].isin(target_class)] 

    y_filtered = selected_rna_data['Label']
    x_filtered = selected_rna_data.drop('Label', axis=1)

    return x_filtered , y_filtered 



### Encode X  - Vanilla One Hot Encoding

In [13]:
number_of_unique_kmers = set()
def encode_seq(kmer_token):

    # A 1 0 0 0
    # C 0 1 0 0
    # T/U 0 0 0 1
    # G 0 0 1 0
    # N 0 0 0 0

    encoding_dict = {
        'A': [1, 0, 0, 0],
        'C': [0, 1, 0, 0],
        'G': [0, 0, 1, 0],
        'T': [0, 0, 0, 1],
        'U': [0, 0, 0, 1],
        'N': [0, 0, 0, 0],
    }

    encoded_sequence = []
    number_of_unique_kmers.add(kmer_token)
    for  base in kmer_token:
        encoded_sequence.append(encoding_dict[base])
    return np.array(encoded_sequence).flatten()

def applyOneHotEncoding(tokenized_sequences):
    encoded_sequences = []
    for seq in tokenized_sequences:
        encoded_sequences.append(encode_seq(seq)) 
 
    return np.array(encoded_sequences).flatten()


def encode_with_one_hot_encoding(x_train_raw):
    truncated_df =  x_train_raw.iloc[:,STRAT_INEDX :END_INDEX] # Window Starts from V501 with 50 window size
    concatenated_column= truncated_df.apply(lambda row: ''.join(map(str, row)), axis=1)
    df_result = truncated_df.assign(Sequence=concatenated_column)
    tokenized_sequences =  df_result['Sequence'].apply(applyOneHotEncoding).tolist()
    return tokenized_sequences


### Encode Y

In [14]:
def encode_target(y_data):
    if TARGET_ENCODING  == 'LABEL':
        le = LabelEncoder()
        y_encoded = le.fit_transform(y_data)
        y_encoded = torch.tensor(y_encoded, dtype=torch.long)

    if TARGET_ENCODING == 'ONE_HOT':
        ohe = OneHotEncoder(sparse=False)
        y_encoded = ohe.fit_transform(y_data.to_numpy().reshape(-1, 1))
        y_encoded = torch.tensor(y_encoded, dtype=torch.float32)

    return y_encoded

### Hyper Parameters

In [15]:
## LSTM Hyper Paramerts 

hyperparameter = {}
hyperparameter['INPUT_DIMENSION'] = 4 # For One Hot Encoding Input Dimension would be 4 as there only 4 unique nucleocide 
hyperparameter['HIDDEN_DIMENSION'] = 256
hyperparameter['NO_OF_LAYERS'] = 3
hyperparameter['BATCH_SIZE'] = 1000
hyperparameter['OUTPUT_DIMENSION'] = 1
hyperparameter['EMBEDDING_DIMENSION'] = 300

### LSTM Model with Binary Classification (Works with Label Encoder)

In [18]:
## Define LSTM and Train

class RNADataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

class RNAClassifier(nn.Module):
    def __init__(self, unqique_k_mer_size,embedding_dim, hidden_dim,num_layers, output_dim):
        super(RNAClassifier, self).__init__()

        #Pytroch Embedding 
        self.embedding = nn.Embedding(unqique_k_mer_size,embedding_dim)

        #LSTM Model
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,num_layers, batch_first=True)

        #Fully Connected Layer   
        self.fc = nn.Linear(hidden_dim, output_dim) 


    def forward(self, x):
        x_embeded =  self.embedding(x)
        lstm_out, (h,c) = self.lstm(x_embeded)
        out = self.fc(h[-1])
        return out.squeeze(1)  # Sequeez to Single dimension for loss calculation

def validate_model(model, test_dataloader , device ,loss_function):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs = inputs.long().to(device)
            labels = labels.float().to(device)

            outputs = model(inputs)
            loss = loss_function(outputs, labels)

            running_loss += loss.item()

            predicted = (outputs > 0.5).float().squeeze()  # Applying Threshold of 0.5 if > then class is 1

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            true_labels.extend(labels.cpu().numpy())  # Capture True Lables for Summary Report
            predicted_labels.extend(predicted.cpu().numpy()) # Capture Predicted Labels Lables for Summary Report

    validation_loss = running_loss / len(test_dataloader)
    validation_accuracy = correct / total

    return validation_loss, validation_accuracy , true_labels , predicted_labels


def train_model(model , train_dataloader , test_dataloader ,device ,epochs, optimizer ,loss_function):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for i, (inputs, labels) in enumerate(train_dataloader):
            
            inputs  = inputs.long().to(device)
            labels = labels.float().to(device)
            optimizer.zero_grad()
            outputs = model(inputs) 
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
        epoch_loss = running_loss / len(train_dataloader)
        val_loss, val_acc , true_labels , predicted_labels= validate_model(model, test_dataloader ,device, loss_function)
        print(f"Epoch {epoch + 1}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")



In [19]:
prediction_class = ['hAm','hCm','hGm','hTm','hm1A','hm5C','hm5U','hm6A','hm6Am','hm7G','hPsi','Atol']

for target  in prediction_class:
    print(f"Running Model Training for class {target}")

    X_train_filtered ,y_train_filtered =  prepare_data_for_binary_classification(x_train_raw,y_train_raw ,target)
    X_test_filtered ,y_test_filtered =  prepare_data_for_binary_classification(x_test_raw,y_test_raw ,target)
    X_valid_filtered ,y_valid_filtered =  prepare_data_for_binary_classification(x_valid_raw,y_valid_raw ,target)

    X_train = torch.tensor(encode_with_one_hot_encoding(X_train_filtered) , dtype=torch.float)
    X_test = torch.tensor(encode_with_one_hot_encoding(X_test_filtered) , dtype=torch.float)
    X_valid = torch.tensor(encode_with_one_hot_encoding(X_valid_filtered) , dtype=torch.float)

    y_train = encode_target(y_train_filtered)
    y_test = encode_target(y_test_filtered)
    y_valid = encode_target(y_valid_filtered)

    sm = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

    unique, counts = np.unique(y_resampled, return_counts=True)
    print(np.asarray((unique, counts)).T)
    

    print("Started Model Training..")

    # Create dataset and dataloader
    train_dataset = RNADataset(X_resampled, y_resampled)
    test_dataset = RNADataset(X_test, y_test)
    valid_dataset = RNADataset(X_valid, y_valid)

    train_dataloader = DataLoader(train_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle=False)
    valid_dataloader = DataLoader(valid_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle=False)



    # Initialize the model, loss function, and optimizer
    model = RNAClassifier(unqique_k_mer_size=hyperparameter['INPUT_DIMENSION'],
                                embedding_dim=hyperparameter['EMBEDDING_DIMENSION'], 
                                hidden_dim=hyperparameter['HIDDEN_DIMENSION'] , 
                                num_layers = hyperparameter['NO_OF_LAYERS'],
                                output_dim=hyperparameter['OUTPUT_DIMENSION'] )

    print(model)
    print("Model Parameters  : " , hyperparameter)

    loss_function = nn.BCEWithLogitsLoss()  ## MSELoss of Regression problem  # BCELoss for binary classification
    optimizer = optim.Adam(model.parameters() ,  lr=0.01)

    # Number of Parameters for Model
    total_parameters = []
    for p in model.parameters():
        total_parameters.append(p.numel())

    print(f"Total Number of Parameters for Model Training : { sum(total_parameters)} " )

    # Train the model
    num_epochs = 10
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Train Model with configured Parameter
    train_model(model, train_dataloader ,test_dataloader, device ,num_epochs,optimizer,loss_function)

    # Evaluate the model on the test dataset
    _, final_accuracy, true_labels, predicted_labels = validate_model(model, valid_dataloader,device,loss_function)

    # Print the final accuracy
    print(f"Class : {target} Final Accuracy: {final_accuracy:.4f}")

    # Print the classification summary
    print("\nClass : {target} Classification Summary:")
    print(classification_report(true_labels, predicted_labels))

    

Running Model Training for class hAm
[[     0 152208]
 [     1 152208]]
Started Model Training..
RNAClassifier(
  (embedding): Embedding(4, 300)
  (lstm): LSTM(300, 256, num_layers=3, batch_first=True)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)
Model Parameters  :  {'INPUT_DIMENSION': 4, 'HIDDEN_DIMENSION': 256, 'NO_OF_LAYERS': 3, 'BATCH_SIZE': 1000, 'OUTPUT_DIMENSION': 1, 'EMBEDDING_DIMENSION': 300}
Total Number of Parameters for Model Training : 1625521 


### Optuna Hyperparameter Training

In [ ]:
# import optuna

# def objective(trial):
#     # Suggest hyperparameters
#     lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
#     embedding_dim = trial.suggest_int("embedding_dim", 10, 300)
#     hidden_dim = trial.suggest_int("hidden_dim", 10, 256)
#     num_layers = trial.suggest_int("num_layers", 1, 3)
#     epochs =  trial.suggest_int("epochs" , 10 , 100)

#     # Initialize the model, loss function, and optimizer with suggested hyperparameters
#     model = RNAClassifier(unqique_k_mer_size=hyperparameter['INPUT_DIMENSION'],
#                           embedding_dim=embedding_dim, 
#                           hidden_dim=hidden_dim, 
#                           num_layers=num_layers,
#                           output_dim=hyperparameter['OUTPUT_DIMENSION'])
    
#     # Check if GPU available the use GPU
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model = model.to(device)

#     loss_function = nn.BCEWithLogitsLoss()
#     optimizer = optim.Adam(model.parameters(), lr=lr)

#     train_model(model, epochs, optimizer , loss_function)

#     validation_loss, final_accuracy, true_labels, predicted_labels = validate_model(model, valid_dataloader,loss_function)
#     # Return the metric you want to optimize (e.g., validation loss or accuracy)
#     return final_accuracy

# # Create Optuna study
# study = optuna.create_study(direction="maximize")  # minimize if optimizing loss, use "maximize" if optimizing accuracy
# study.optimize(objective, n_trials=50)

# # Print best hyperparameters
# print("Best hyperparameters:", study.best_params)


[I 2023-05-05 14:53:18,775] A new study created in memory with name: no-name-158c73bb-0438-4edf-9714-b8f319c1c534
C:\Users\shashi.vish\AppData\Local\Temp\ipykernel_35172\835731868.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)


### LSTM Model with Binary Cross Entropy Loss (Works with One Hot Encoder)

In [ ]:
# ## Define LSTM and Train

# class RNADataset(Dataset):
#     def __init__(self, X, y):
#         self.X = X
#         self.y = y

#     def __len__(self):
#         return len(self.X)

#     def __getitem__(self, index):
#         return self.X[index], self.y[index]

# class RNAClassifier_BCE(nn.Module):
#     def __init__(self, unqique_k_mer_size, embedding_dim,hidden_dim,num_layers, output_dim):
#         super(RNAClassifier_BCE, self).__init__()
#         self.embedding  = nn.Embedding(unqique_k_mer_size,embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, output_dim)

#     def forward(self, x):

#         x_embeded = self.embedding(x)
#         _, (hidden, _) = self.lstm(x_embeded)

#        # print("Hidden Dimansion : " , hidden.shape)
#         x = self.fc(hidden[-1]) # Squeeze does not make any differecent in Tensor , basically removing 0 demsion from tensor 

#         # When we squeeze a tensor, the dimensions of size 1 are removed. 
#         # The elements of the original tensor are arranged with the remaining dimensions. For example, 
#         # if the input tensor is of shape: (m×1×n×1) then the output tensor after squeeze will be of shape: (m×n). 

#         # print("Ouput  Dimansion : " ,x.shape)      

#         # print("Result After Applygin Softmax : " , torch.softmax(x, dim=-1) )                                      
#         return x



# if MODEL==1 :
#     print("Excuting LSTM Model")

#     # Create dataset and dataloader
#     print("X Shape of Tensor : " , X_train.shape , " Y Tensor Shape : " , y_train.shape)
#     print("HyperParameter for Training : ", hyperparameter)
#     train_dataset = RNADataset(X_train, y_train)
#     test_dataset = RNADataset(X_test, y_test)

#     train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
#     test_dataloader = DataLoader(test_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle=False)

#     # Calculating INPUT_DIMENSION for LSTM Mode 
#     # If you are just using One Hot then INPUT_DIMENSION would be 4 because there are 4 possible unique nucleosides
#     # But if you are using K mers then INPUT_DIMENSION would be number of unique k-mers on all dataset

#     # LSTM model Format - (batch_size, sequence_length, input_size)  
#     # batch_size comes from DataLoaded , you can tune it.
#     # Sequence Length - Lenth of X[0] shape (32, 19, 12)  Here 19 is sequence length , 
#     # Input Size = 12 length of X[0][0] - Basically all unique nucleoside or k-mers

#     # Initialize the model, loss function, and optimizer
#     # model = RNAClassifier_BCE(input_size=4788, 
#     #                           hidden_size=hyperparameter['HIDDEN_DIMENSION'] , 
#     #                           num_layers = hyperparameter['NO_OF_LAYERS'],
#     #                           output_size=hyperparameter['OUTPUT_DIMENSION'] )
#     model = RNAClassifier_BCE(unqique_k_mer_size=hyperparameter['INPUT_DIMENSION'],
#                               embedding_dim=hyperparameter['EMBEDDING_DIMENSION'] 
#                               hidden_dim=hyperparameter['HIDDEN_DIMENSION'] , 
#                               num_layers = hyperparameter['NO_OF_LAYERS'],
#                               output_dim=hyperparameter['OUTPUT_DIMENSION'] )
    
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters())

#     # Number of Parameters for Model
#     total_parameters = []
#     for p in model.parameters():
#         total_parameters.append(p.numel())
    
#     print(f"Total Number of Parameters for Model Training : { sum(total_parameters)} " )

#     # Train the model
#     num_epochs = 10
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model = model.to(device)
    
#     for epoch in range(num_epochs):
#         model.train()
#         running_loss = 0.0
#         correct = 0
#         total = 0

#         for i, (inputs, labels) in enumerate(train_dataloader):
#             inputs, labels = inputs.to(device), labels.to(device)

#             print("Target Labesl : " , labels.shape)
#             print("Ouput Labesl : " , outputs.shape)

#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()

#             running_loss += loss.item()
            
#         epoch_loss = running_loss / len(train_dataloader)
#         print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")




SyntaxError: invalid syntax. Perhaps you forgot a comma? (1466345586.py, line 67)

In [ ]:
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for x_batch, y_batch in test_dataloader:
        outputs = model(x_batch)
        
        # Basically models predict probability for each class and then finally we pick up with hightest probabiliyt class 
        # and then compare with actual class to calculate accuracy. 
        
        ## Predicted torch.max(outputs, 1) 
#         values=tensor([0.7304, 0.4554, 0.3984, 0.4091, 0.9271, 0.4370, 0.3837, 0.4395, 0.4005,
#         0.5451, 0.5642, 0.6866, 0.5768, 0.5160, 0.5093, 0.4947, 0.6823, 0.4612,
#         0.4073, 0.5041, 0.6982, 0.4116, 0.4092, 0.5589, 0.6010, 0.4693, 0.6324,
#         0.5524, 0.4339, 0.7889, 0.5216, 0.6588]),
#         indices=tensor([ 1, 10, 10, 10,  1, 10, 10, 10,  1, 10,  1,  1,  0,  1,  1, 10,  1, 10,
#         10, 10, 10,  1, 10,  0, 10,  1,  1,  0, 10,  1,  1,  0]))

        _, predicted = torch.max(outputs, 1)  # Get the class index with the highest probability
        #print(torch.max(outputs, 1))
        true_labels = torch.max(y_batch, 1).indices  # Get the true class indices from the one-hot encoded labels [0,1,0,0,,,... ] Get index of 1
        correct_predictions += (predicted == true_labels).sum().item()
        total_predictions += y_batch.size(0)

accuracy = correct_predictions / total_predictions


print(f"Validation Accuracy: {accuracy:.4f}")


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)